# 目前不需要但是testing data 可以參考作法

In [12]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

X_val=pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test.csv')# 讀取X_test.csv
X_train=pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')# 讀取X_train.csv
Y_train=pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')# 讀取Y_train.csv
# If y_train is loaded as DataFrame, convert to Series
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# Scale features(特徵縮放)
print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) 

print("Scaling features...")
scaler = StandardScaler()
X_val_scaled = scaler.fit_transform(X_val)

Scaling features...
Scaling features...


# 目前不需要，先留著

In [13]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import os

# 1. 取得y_test的資料筆數
y_val = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test.csv')
val_size = len(y_val)
print(f"驗證資料筆數: {val_size}")

# 2. 使用Windows路徑
save_dir = r'C:\Users\User\.vscode\thesis_experiment\CIC-DDoS-2019\01-12'
os.makedirs(save_dir, exist_ok=True)

# 3. 定義所有模型
models = {
    'SVM': SVC(kernel='rbf', random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# 4. 訓練模型並生成預測結果
for name, model in tqdm(models.items(), desc="生成預測結果"):
    print(f"\n處理 {name}...")
    
    # 訓練模型
    model.fit(X_train_scaled, y_train) 
    
    # 生成預測結果
    y_pred = model.predict(X_val_scaled) #透過把訓練資料丟進去給模型，模型會生成出一個對應的預測結果(長度會跟y_val 一樣，以便比較模型是否過擬合) 
    
    # 儲存預測結果
    save_path = os.path.join(save_dir, f'{name.replace(" ", "_")}_y_pred_val.csv')
    pd.DataFrame(y_pred, columns=['prediction']).to_csv(save_path, index=False)
    print(f"已儲存 {name} 預測結果，筆數: {len(y_pred)}")

驗證資料筆數: 109521


生成預測結果:   0%|          | 0/3 [00:00<?, ?it/s]


處理 SVM...


生成預測結果:  33%|███▎      | 1/3 [04:58<09:56, 298.07s/it]

已儲存 SVM 預測結果，筆數: 109521

處理 Logistic Regression...


生成預測結果:  67%|██████▋   | 2/3 [05:01<02:04, 124.67s/it]

已儲存 Logistic Regression 預測結果，筆數: 109521

處理 KNN...


生成預測結果: 100%|██████████| 3/3 [05:36<00:00, 112.20s/it]

已儲存 KNN 預測結果，筆數: 109521


# "用validation data, training data 來個別驗證經過自適應往格搜尋調整過後的SVM 模型的訓練及驗證準確(檢查模型是否會過擬合)"

In [1]:
import os
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import time

# 添加自適應參數範圍優化類
class AdaptiveGridSearch:
    def __init__(self, estimator, param_grid, cv=5, scoring='accuracy', n_iterations=3):
        """
        自適應參數範圍優化器
        
        參數:
        estimator: 模型估計器
        param_grid: 初始參數網格
        cv: 交叉驗證折數
        scoring: 評分標準
        n_iterations: 迭代優化次數
        """
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_score_ = 0
        self.best_estimator_ = None
        self.optimization_history = []
        
    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        """根據當前最佳參數生成新的搜索範圍"""
        new_param_grid = {}
        shrink_factor = 0.5 / (iteration + 1)  # 範圍收縮因子，隨迭代次數增加而減小
        
        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                # 對於分類參數(如kernel)，直接使用最佳值
                new_param_grid[param_name] = [param_value]
                continue
                
            current_values = param_grid[param_name]
            
            if len(current_values) <= 1:
                new_param_grid[param_name] = current_values
                continue
                
            # 找出當前最佳值在參數列表中的位置
            try:
                idx = current_values.index(param_value)
            except ValueError:
                # 如果最佳值不在列表中，找最接近的值
                idx = min(range(len(current_values)), 
                         key=lambda i: abs(current_values[i] - param_value))
            
            # 計算新的參數範圍
            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor
            
            if param_range < 1e-10:  # 如果範圍已經非常小，則不再縮小
                new_param_grid[param_name] = current_values
                continue
                
            # 根據參數類型決定如何生成新值
            if isinstance(param_value, int):
                # 整數參數
                lower = max(int(param_value - delta), min(current_values))
                upper = min(int(param_value + delta), max(current_values))
                
                # 確保至少有3個值
                if lower == upper:
                    new_values = [lower]
                else:
                    step = max(1, (upper - lower) // 3)
                    new_values = list(range(lower, upper + 1, step))
            else:
                # 浮點數參數
                lower = max(param_value - delta, min(current_values))
                upper = min(param_value + delta, max(current_values))
                
                # 生成更精細的網格
                step = (upper - lower) / 3
                new_values = [round(lower + i * step, 6) for i in range(4)]  # 包括上界
            
            # 確保參數值的獨特性和排序
            new_param_grid[param_name] = sorted(list(set(new_values)))
            
        return new_param_grid
    
    def fit(self, X, y):
        """執行自適應網格搜索"""
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        
        print("開始自適應參數範圍優化...")
        
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            
            # 執行當前迭代的GridSearchCV
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=self.cv, 
                scoring=self.scoring,
                n_jobs=-1
            )
            
            grid_search.fit(X, y)
            
            # 更新最佳參數和分數
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            
            # 記錄這次迭代的結果
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score
            })
            
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            
            # 更新全局最佳結果
            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            
            # 為下一次迭代生成新的參數網格
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        
        # 打印優化總結
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        
        # 展示優化過程
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}")
        
        return self

# 1. 資料載入
print("Loading data...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_val = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test.csv')
y_val = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test.csv')

# 將 Y_train 轉換為 Series
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 2. 特徵縮放
print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 3. 自適應參數範圍優化 (針對 SVM)
print("\n開始SVM超參數自適應優化...")
# 初始參數網格定義
param_grid = {
    'C': [80, 90, 100, 110, 120],              # 擴大初始範圍以便更好地探索
    'gamma': [0.008, 0.009, 0.01, 0.011, 0.012], # 擴大初始範圍
    'kernel': ['rbf']
}

# 使用自適應網格搜索替代傳統GridSearchCV
svm_model = SVC(random_state=42)
adaptive_search = AdaptiveGridSearch(
    svm_model, 
    param_grid, 
    cv=5, 
    scoring='accuracy',
    n_iterations=3  # 執行3次迭代優化
)

# 執行自適應搜索
adaptive_search.fit(X_train_scaled, y_train)

# 獲取最佳參數和最佳模型
best_svm_model = adaptive_search.best_estimator_
best_params = adaptive_search.best_params_
print(f'最終最佳SVM參數: {best_params}')

# 計算最佳 SVM 模型的訓練準確率
train_accuracy_svm = best_svm_model.score(X_train_scaled, y_train)
print(f'SVM training accuracy: {train_accuracy_svm:.6f}')

# 4. 其他模型訓練與評估 (Logistic Regression, KNN)
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

print("\nTraining Other Models...")
for name, model in tqdm(models.items()):
    print(f'Training {name}...')
    model.fit(X_train_scaled, y_train)
    print(f'{name} training complete!')
    train_accuracy = model.score(X_train_scaled, y_train)
    print(f'{name} validation Accuracy: {train_accuracy:.6f}')

# 5. 測試集評估 (所有模型)
print("\nEvaluating Models on validation Set...")

# 6. 定義評估指標
def evaluate_model(model, X_val, y_val, model_name="Model"):
    """評估模型性能，並輸出結果。"""
    # 預測測試集
    y_pred_val = model.predict(X_val_scaled) #透過把validation data丟進去給經過自適應往格搜尋的模型，模型會生成一個對應的預測結果(長度會跟y_val 一樣，以便比較模型是否過擬合)

    # 計算準確率
    accuracy = accuracy_score(y_val, y_pred_val)
    print(f"{model_name} (Accuracy): {accuracy:.6f}")

    # 產生分類報告
    report = classification_report(y_val, y_pred_val)
    print(f"{model_name}  (Classification Report):\n{report}")

    # 計算混淆矩陣
    cm = confusion_matrix(y_val, y_pred_val)
    print(f"{model_name}  (Confusion Matrix):\n{cm}")
    
    # 返回關鍵指標用於比較
    return {
        'validation accuracy': accuracy,
        'validation report': report,
        'validation confusion_matrix': cm
    }

# 7. 評估每個模型
all_models = {
    'SVM (Adaptive)': best_svm_model,
    'Logistic Regression': models['Logistic Regression'],
    'KNN': models['KNN']
}

print("\n============ The comparison of model evaluation  ============")
results = {}
for name, model in all_models.items():
    print(f"\n============ {name} model evaluation result ============")
    results[name] = evaluate_model(model, X_val_scaled, y_val, model_name=name)
    print("===========================================")

# 8. 比較模型性能
print("\n============ 模型性能比較 ============")
for name, result in results.items():
    print(f"{name} 模型驗證準確率: {result['validation accuracy']:.6f}")

Loading data...
Scaling features...

開始SVM超參數自適應優化...
開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [80, 90, 100, 110, 120], 'gamma': [0.008, 0.009, 0.01, 0.011, 0.012], 'kernel': ['rbf']}
迭代 1 最佳參數: {'C': 100, 'gamma': 0.009, 'kernel': 'rbf'}
迭代 1 最佳分數: 0.999666

迭代 2/3
當前參數搜索範圍: {'C': [80, 93, 106, 119], 'gamma': [0.008, 0.009, 0.01, 0.011], 'kernel': ['rbf']}
迭代 2 最佳參數: {'C': 106, 'gamma': 0.009, 'kernel': 'rbf'}
迭代 2 最佳分數: 0.999666

迭代 3/3
當前參數搜索範圍: {'C': [96, 102, 108, 114], 'gamma': [0.00825, 0.00875, 0.00925, 0.00975], 'kernel': ['rbf']}
迭代 3 最佳參數: {'C': 102, 'gamma': 0.00875, 'kernel': 'rbf'}
迭代 3 最佳分數: 0.999666

自適應參數範圍優化完成!
總耗時: 4009.08 秒
最終最佳參數: {'C': 100, 'gamma': 0.009, 'kernel': 'rbf'}
最終最佳分數: 0.999666

參數優化歷史:
迭代 1: 分數=0.999666, 參數={'C': 100, 'gamma': 0.009, 'kernel': 'rbf'}
迭代 2: 分數=0.999666, 參數={'C': 106, 'gamma': 0.009, 'kernel': 'rbf'}
迭代 3: 分數=0.999666, 參數={'C': 102, 'gamma': 0.00875, 'kernel': 'rbf'}
最終最佳SVM參數: {'C': 100, 'gamma': 0.009, 'kernel': 'rbf'}
SVM training accu

  0%|          | 0/2 [00:00<?, ?it/s]

Training Logistic Regression...


 50%|█████     | 1/2 [00:02<00:02,  2.72s/it]

Logistic Regression training complete!
Logistic Regression validation Accuracy: 0.989542
Training KNN...
KNN training complete!


100%|██████████| 2/2 [01:56<00:00, 58.27s/it]

KNN validation Accuracy: 0.999767

Evaluating Models on validation Set...

============ The comparison of model evaluation  ============

============ SVM (Adaptive) model evaluation result ============


SVM (Adaptive) (Accuracy): 0.999690
SVM (Adaptive)  (Classification Report):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18807
           1       1.00      1.00      1.00     90714

    accuracy                           1.00    109521
   macro avg       1.00      1.00      1.00    109521
weighted avg       1.00      1.00      1.00    109521

SVM (Adaptive)  (Confusion Matrix):
[[18792    15]
 [   19 90695]]

============ Logistic Regression model evaluation result ============
Logistic Regression (Accuracy): 0.985793
Logistic Regression  (Classification Report):
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     18807
           1       1.00      0.98      0.99     90714

    accuracy                           0.99    109521
   macro avg       0.96      0.99      0.98    109521
weighted avg       0.99      0.99      0.99    109521

Logistic Regression  (Confusion Matrix):
[[1

# "用validation data, training data  來個別驗證經過自適應往格搜尋調整過後的KNN模型的訓練及驗證準確(檢查模型是否會過擬合)"

In [1]:
import os
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import time

# 添加自適應參數範圍優化類
class AdaptiveGridSearch:
    def __init__(self, estimator, param_grid, cv=5, scoring='accuracy', n_iterations=3):
        """
        自適應參數範圍優化器
        
        參數:
        estimator: 模型估計器
        param_grid: 初始參數網格
        cv: 交叉驗證折數
        scoring: 評分標準
        n_iterations: 迭代優化次數
        """
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_score_ = 0
        self.best_estimator_ = None
        self.optimization_history = []
        
    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        """根據當前最佳參數生成新的搜索範圍"""
        new_param_grid = {}
        shrink_factor = 0.5 / (iteration + 1)  # 範圍收縮因子，隨迭代次數增加而減小
        
        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                # 對於分類參數(如algorithm)，直接使用最佳值
                new_param_grid[param_name] = [param_value]
                continue
                
            current_values = param_grid[param_name]
            
            if len(current_values) <= 1:
                new_param_grid[param_name] = current_values
                continue
                
            # 找出當前最佳值在參數列表中的位置
            try:
                idx = current_values.index(param_value)
            except ValueError:
                # 如果最佳值不在列表中，找最接近的值
                idx = min(range(len(current_values)), 
                         key=lambda i: abs(current_values[i] - param_value))
            
            # 計算新的參數範圍
            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor
            
            if param_range < 1e-10:  # 如果範圍已經非常小，則不再縮小
                new_param_grid[param_name] = current_values
                continue
                
            # 根據參數類型決定如何生成新值
            if isinstance(param_value, int):
                # 整數參數
                lower = max(int(param_value - delta), min(current_values))
                upper = min(int(param_value + delta), max(current_values))
                
                # 確保至少有3個值
                if lower == upper:
                    new_values = [lower]
                else:
                    step = max(1, (upper - lower) // 3)
                    new_values = list(range(lower, upper + 1, step))
            else:
                # 浮點數參數
                lower = max(param_value - delta, min(current_values))
                upper = min(param_value + delta, max(current_values))
                
                # 生成更精細的網格
                step = (upper - lower) / 3
                new_values = [round(lower + i * step, 6) for i in range(4)]  # 包括上界
            
            # 確保參數值的獨特性和排序
            new_param_grid[param_name] = sorted(list(set(new_values)))
            
        return new_param_grid
    
    def fit(self, X, y):
        """執行自適應網格搜索"""
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        
        print("開始自適應參數範圍優化...")
        
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            
            # 執行當前迭代的GridSearchCV
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=self.cv, 
                scoring=self.scoring,
                n_jobs=-1
            )
            
            grid_search.fit(X, y)
            
            # 更新最佳參數和分數
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            
            # 記錄這次迭代的結果
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score
            })
            
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            
            # 更新全局最佳結果
            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            
            # 為下一次迭代生成新的參數網格
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        
        # 打印優化總結
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        
        # 展示優化過程
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}")
        
        return self

# 1. 資料載入
print("Loading data...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_val = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test.csv')
y_val = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test.csv')


# 將 Y_train 轉換為 Series
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 2. 特徵縮放
print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 3. 對KNN進行自適應參數範圍優化
print("\n開始KNN超參數自適應優化...")
# 初始參數網格定義
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # p=1 為曼哈頓距離，p=2 為歐氏距離
}

# 使用自適應網格搜索
knn_model = KNeighborsClassifier()
adaptive_search_knn = AdaptiveGridSearch(
    knn_model,
    knn_param_grid,
    cv=5,
    scoring='accuracy',
    n_iterations=3
)

# 執行自適應搜索
adaptive_search_knn.fit(X_train_scaled, y_train)

# 獲取最佳參數和最佳模型
best_knn_model = adaptive_search_knn.best_estimator_
best_knn_params = adaptive_search_knn.best_params_
print(f'最終最佳KNN參數: {best_knn_params}')

# 計算最佳 KNN 模型的訓練準確率
train_accuracy_knn = best_knn_model.score(X_train_scaled, y_train)
print(f'KNN training accuracy: {train_accuracy_knn:.6f}')

# 4. 定義評估指標
def evaluate_model(model, X_val, y_val, model_name="Model"):
    """評估模型性能，並輸出結果。"""
    # 預測測試集
    y_pred_val = model.predict(X_val_scaled)

    # 計算準確率
    accuracy = accuracy_score(y_val, y_pred_val)
    print(f"{model_name} Validation Accuracy: {accuracy:.6f}")

    # 產生分類報告
    report = classification_report(y_val, y_pred_val)
    print(f"{model_name} Classification Report:\n{report}")

    # 計算混淆矩陣
    cm = confusion_matrix(y_val, y_pred_val)
    print(f"{model_name} Confusion Matrix:\n{cm}")
    
    # 返回關鍵指標用於比較
    return {
        'accuracy': accuracy,
        'report': report,
        'confusion_matrix': cm
    }

# 5. 評估最佳KNN模型
print("\n============ KNN (Adaptive) model evaluation result ============")
knn_results = evaluate_model(best_knn_model, X_val_scaled, y_val, model_name="KNN (Adaptive)")
print("===========================================")

# 6. 打印KNN的最佳參數
print("\n============ KNN最佳參數總結 ============")
print(f"KNN最佳參數: {best_knn_params}")
print(f"KNN最佳驗證準確率: {knn_results['accuracy']:.6f}")

Loading data...
Scaling features...

開始KNN超參數自適應優化...
開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'p': [1, 2]}
迭代 1 最佳參數: {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 1 最佳分數: 0.999799

迭代 2/3
當前參數搜索範圍: {'algorithm': ['auto'], 'n_neighbors': [3, 4, 5, 6, 7], 'p': [1], 'weights': ['distance']}
迭代 2 最佳參數: {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 2 最佳分數: 0.999799

迭代 3/3
當前參數搜索範圍: {'algorithm': ['auto'], 'n_neighbors': [3, 4], 'p': [1], 'weights': ['distance']}
迭代 3 最佳參數: {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 3 最佳分數: 0.999799

自適應參數範圍優化完成!
總耗時: 24248.85 秒
最終最佳參數: {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
最終最佳分數: 0.999799

參數優化歷史:
迭代 1: 分數=0.999799, 參數={'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 2: 分數=0.999799, 參數={'algorit

# "用validation data, training data  來個別驗證經過自適應往格搜尋調整過後的LR模型的訓練及驗證準確(檢查模型是否會過擬合)"

In [2]:
import os
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import time

# 自適應參數範圍優化類
class AdaptiveGridSearch:
    def __init__(self, estimator, param_grid, cv=5, scoring='accuracy', n_iterations=3):
        """
        自適應參數範圍優化器
        
        參數:
        estimator: 模型估計器
        param_grid: 初始參數網格
        cv: 交叉驗證折數
        scoring: 評分標準
        n_iterations: 迭代優化次數
        """
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_score_ = 0
        self.best_estimator_ = None
        self.optimization_history = []
        
    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        """根據當前最佳參數生成新的搜索範圍"""
        new_param_grid = {}
        shrink_factor = 0.5 / (iteration + 1)  # 範圍收縮因子，隨迭代次數增加而減小
        
        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                # 對於分類參數(如solver, penalty)，直接使用最佳值
                new_param_grid[param_name] = [param_value]
                continue
                
            current_values = param_grid[param_name]
            
            if len(current_values) <= 1:
                new_param_grid[param_name] = current_values
                continue
                
            # 找出當前最佳值在參數列表中的位置
            try:
                idx = current_values.index(param_value)
            except ValueError:
                # 如果最佳值不在列表中，找最接近的值
                idx = min(range(len(current_values)), 
                         key=lambda i: abs(current_values[i] - param_value))
            
            # 計算新的參數範圍
            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor
            
            if param_range < 1e-10:  # 如果範圍已經非常小，則不再縮小
                new_param_grid[param_name] = current_values
                continue
                
            # 根據參數類型決定如何生成新值
            if isinstance(param_value, int):
                # 整數參數
                lower = max(int(param_value - delta), min(current_values))
                upper = min(int(param_value + delta), max(current_values))
                
                # 確保至少有3個值
                if lower == upper:
                    new_values = [lower]
                else:
                    step = max(1, (upper - lower) // 3)
                    new_values = list(range(lower, upper + 1, step))
            else:
                # 浮點數參數
                lower = max(param_value - delta, min(current_values))
                upper = min(param_value + delta, max(current_values))
                
                # 生成更精細的網格
                step = (upper - lower) / 3
                new_values = [round(lower + i * step, 6) for i in range(4)]  # 包括上界
            
            # 確保參數值的獨特性和排序
            new_param_grid[param_name] = sorted(list(set(new_values)))
            
        return new_param_grid
    
    def fit(self, X, y):
        """執行自適應網格搜索"""
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        
        print("開始自適應參數範圍優化...")
        
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            
            # 執行當前迭代的GridSearchCV
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=self.cv, 
                scoring=self.scoring,
                n_jobs=-1,
                verbose=0  # 減少輸出
            )
            
            grid_search.fit(X, y)
            
            # 更新最佳參數和分數
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            
            # 記錄這次迭代的結果
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score
            })
            
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            
            # 更新全局最佳結果
            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            
            # 為下一次迭代生成新的參數網格
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        
        # 打印優化總結
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        
        # 展示優化過程
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}")
        
        return self

# 1. 資料載入
print("Loading data...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_val = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test.csv')
y_val = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test.csv')

# 將 Y_train 轉換為 Series
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 2. 特徵縮放
print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 3. 對LR進行自適應參數範圍優化
print("\n開始LR超參數自適應優化...")

# 定義更新過的參數網格
lr_param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0, 100.0],  # 更新了C的範圍
    'solver': ['liblinear', 'saga'],
    'max_iter': [200, 500],
    'tol': [1e-4]
}

# 使用自適應網格搜索
adaptive_search_lr = AdaptiveGridSearch(
    LogisticRegression(random_state=42),
    lr_param_grid,
    cv=3,  # 減少交叉驗證折數以加速
    scoring='accuracy',
    n_iterations=3  # 設定迭代次數
)

# 執行自適應網格搜索
adaptive_search_lr.fit(X_train_scaled, y_train)
best_lr_model = adaptive_search_lr.best_estimator_
best_lr_params = adaptive_search_lr.best_params_
best_lr_score = adaptive_search_lr.best_score_

# 4. 定義評估指標
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name="Model"):
    """評估模型性能，並輸出結果。"""
    # 預測訓練集
    y_pred_train = model.predict(X_train)
    # 預測測試集
    y_pred_val = model.predict(X_val)

    # 計算訓練和驗證準確率
    train_accuracy = accuracy_score(y_train, y_pred_train)
    val_accuracy = accuracy_score(y_val, y_pred_val)

    print(f"{model_name} Training Accuracy: {train_accuracy:.6f}")
    print(f"{model_name} Validation Accuracy: {val_accuracy:.6f}")

    # 產生分類報告
    report = classification_report(y_val, y_pred_val)
    print(f"{model_name} Classification Report:\n{report}")

    # 計算混淆矩陣
    cm = confusion_matrix(y_val, y_pred_val)
    print(f"{model_name} Confusion Matrix:\n{cm}")
    
    # 返回關鍵指標用於比較
    return {
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy,
        'report': report,
        'confusion_matrix': cm
    }

# 5. 評估最佳LR模型
print("\n============ LR (Adaptive) model evaluation result ============")
lr_results = evaluate_model(best_lr_model, X_train_scaled, y_train, X_val_scaled, y_val, model_name="LR (Adaptive)")
print("===========================================")

# 6. 打印LR的最佳參數
print("\n============ LR最佳參數總結 ============")
print(f"LR最佳參數: {best_lr_params}")
print(f"LR最佳驗證準確率: {lr_results['val_accuracy']:.6f}")


Loading data...
Scaling features...

開始LR超參數自適應優化...
開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [0.01, 0.1, 1.0, 10.0, 100.0], 'solver': ['liblinear', 'saga'], 'max_iter': [200, 500], 'tol': [0.0001]}


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


迭代 1 最佳參數: {'C': 0.01, 'max_iter': 500, 'solver': 'saga', 'tol': 0.0001}
迭代 1 最佳分數: 0.990592

迭代 2/3
當前參數搜索範圍: {'C': [0.01, 16.675, 33.34, 50.005], 'max_iter': [350, 400, 450, 500], 'solver': ['saga'], 'tol': [0.0001]}


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


迭代 2 最佳參數: {'C': 0.01, 'max_iter': 400, 'solver': 'saga', 'tol': 0.0001}
迭代 2 最佳分數: 0.990616

迭代 3/3
當前參數搜索範圍: {'C': [0.01, 4.17625, 8.3425, 12.50875], 'max_iter': [362, 387, 412, 437], 'solver': ['saga'], 'tol': [0.0001]}
迭代 3 最佳參數: {'C': 0.01, 'max_iter': 437, 'solver': 'saga', 'tol': 0.0001}
迭代 3 最佳分數: 0.990616

自適應參數範圍優化完成!
總耗時: 1731.65 秒
最終最佳參數: {'C': 0.01, 'max_iter': 400, 'solver': 'saga', 'tol': 0.0001}
最終最佳分數: 0.990616

參數優化歷史:
迭代 1: 分數=0.990592, 參數={'C': 0.01, 'max_iter': 500, 'solver': 'saga', 'tol': 0.0001}
迭代 2: 分數=0.990616, 參數={'C': 0.01, 'max_iter': 400, 'solver': 'saga', 'tol': 0.0001}
迭代 3: 分數=0.990616, 參數={'C': 0.01, 'max_iter': 437, 'solver': 'saga', 'tol': 0.0001}

============ LR (Adaptive) model evaluation result ============
LR (Adaptive) Training Accuracy: 0.990638
LR (Adaptive) Validation Accuracy: 0.986496
LR (Adaptive) Classification Report:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     18807
          

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# "用tesing data(之前模型都沒有看過的測試資料(資料夾03-11)來個別測試經經過自適應往格搜尋調整過後的SVM模型的測試準確率(模型最終的表現))"

# 

In [22]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

# ===============================
# 自適應參數範圍優化類別（強化版）
# ===============================
class AdaptiveGridSearch:
    def __init__(self, estimator, param_grid, cv=5, scoring='accuracy', n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_score_ = 0
        self.best_estimator_ = None
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        """根據最佳參數縮小搜索範圍"""
        new_param_grid = {}
        shrink_factor = 0.5 / (iteration + 1)

        for param, best_val in best_params.items():
            if isinstance(param_grid[param][0], str):
                new_param_grid[param] = [best_val]
                continue

            values = param_grid[param]
            param_range = max(values) - min(values)
            delta = param_range * shrink_factor

            if isinstance(best_val, int):
                lower = max(min(values), int(best_val - delta))
                upper = min(max(values), int(best_val + delta))
                step = max(1, (upper - lower) // 3)
                new_values = list(range(lower, upper + 1, step))
            else:
                lower = max(min(values), best_val - delta)
                upper = min(max(values), best_val + delta)
                step = (upper - lower) / 3
                new_values = [round(lower + i * step, 6) for i in range(4)]

            new_param_grid[param] = sorted(list(set(new_values)))

        return new_param_grid

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid

        print("開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"參數搜索範圍: {current_param_grid}")

            grid = GridSearchCV(
                self.estimator,
                current_param_grid,
                cv=self.cv,
                scoring=self.scoring,
                n_jobs=-1
            )
            grid.fit(X, y)

            best_params = grid.best_params_
            best_score = grid.best_score_

            self.optimization_history.append({
                'iteration': i + 1,
                'param_grid': current_param_grid,
                'best_params': best_params,
                'best_score': best_score
            })

            if i == 0 or best_score > self.best_score_:
                self.best_params_ = best_params
                self.best_score_ = best_score
                self.best_estimator_ = grid.best_estimator_

            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    best_params, current_param_grid, i)

        elapsed_time = time.time() - start_time
        print("\n自適應參數優化完成")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最佳參數: {self.best_params_}")
        print(f" 最佳交叉驗證分數: {self.best_score_:.6f}")

        return self



print("載入資料...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

print("特徵標準化...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===============================
# SVM + 自適應網格搜尋
# ===============================
print("執行 SVM 自適應搜尋...")
initial_param_grid = {
    'C': [1, 10, 100, 500, 1000],  # 起手範圍拉大
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

svm = SVC(random_state=42)
adaptive_search = AdaptiveGridSearch(
    estimator=svm,
    param_grid=initial_param_grid,
    cv=5,
    scoring='accuracy',
    n_iterations=3
)

adaptive_search.fit(X_train_scaled, y_train)
best_svm_model = adaptive_search.best_estimator_

# ===============================
# 測試集評估
# ===============================
print("\n 模型測試集評估")
y_pred = best_svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"測試準確率: {accuracy:.6f}")
print("\n分類報告:")
print(classification_report(y_test, y_pred))
print("\n混淆矩陣:")
print(confusion_matrix(y_test, y_pred))


載入資料...
特徵標準化...
執行 SVM 自適應搜尋...
開始自適應參數範圍優化...

迭代 1/3
參數搜索範圍: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}

迭代 2/3
參數搜索範圍: {'C': [500, 666, 832, 998], 'gamma': [0.0001, 0.02005, 0.04, 0.05995], 'kernel': ['rbf']}

迭代 3/3
參數搜索範圍: {'C': [873, 914, 955, 996], 'gamma': [0.044987, 0.049975, 0.054963, 0.05995], 'kernel': ['rbf']}

自適應參數優化完成
總耗時: 12899.12 秒
最佳參數: {'C': 996, 'gamma': 0.049975, 'kernel': 'rbf'}
 最佳交叉驗證分數: 0.999702

 模型測試集評估
測試準確率: 0.998408

分類報告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49763
           1       1.00      1.00      1.00     25620

    accuracy                           1.00     75383
   macro avg       1.00      1.00      1.00     75383
weighted avg       1.00      1.00      1.00     75383


混淆矩陣:
[[49646   117]
 [    3 25617]]


有調整過的自適應超參數調整

In [7]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

# 1. 載入資料
print("Loading data...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test.csv')

# Y_train 轉為 Series
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 2. 特徵縮放
print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. 自適應網格搜尋類別
class AdaptiveGridSearch:
    def __init__(self, estimator, param_grid, cv=5, scoring='accuracy', n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_score_ = 0
        self.best_estimator_ = None
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        shrink_factor = 0.5 / (iteration + 1)
        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                new_param_grid[param_name] = [param_value]
                continue
            current_values = param_grid[param_name]
            if len(current_values) <= 1:
                new_param_grid[param_name] = current_values
                continue
            try:
                idx = current_values.index(param_value)
            except ValueError:
                idx = min(range(len(current_values)), key=lambda i: abs(current_values[i] - param_value))
            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor
            if param_range < 1e-10:
                new_param_grid[param_name] = current_values
                continue
            if isinstance(param_value, int):
                lower = max(int(param_value - delta), min(current_values))
                upper = min(int(param_value + delta), max(current_values))
                if lower == upper:
                    new_values = [lower]
                else:
                    step = max(1, (upper - lower) // 3)
                    new_values = list(range(lower, upper + 1, step))
            else:
                lower = max(param_value - delta, min(current_values))
                upper = min(param_value + delta, max(current_values))
                step = (upper - lower) / 3
                new_values = [round(lower + i * step, 6) for i in range(4)]
            new_param_grid[param_name] = sorted(list(set(new_values)))
        return new_param_grid

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        print("開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            iter_start = time.time()
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=self.cv, 
                scoring=self.scoring,
                n_jobs=-1
            )
            grid_search.fit(X, y)
            iter_end = time.time()
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_end - iter_start
            })
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_end - iter_start:.2f} 秒")
            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}, 耗時={history['iteration_time']:.2f} 秒")
        return self

# 4. 定義參數範圍
param_grid = {
    'C': [1, 10, 100, 500, 1000],
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

# 5. 執行自適應搜尋（含總訓練耗時）
svm_model = SVC(random_state=42)
adaptive_search = AdaptiveGridSearch(
    svm_model, 
    param_grid, 
    cv=5, 
    scoring='accuracy',
    n_iterations=3
)

total_train_start = time.time()
adaptive_search.fit(X_train_scaled, y_train)
total_train_end = time.time()
print(f"\nSVM自適應網格搜尋總訓練耗時: {total_train_end - total_train_start:.2f} 秒")

# 6. 測試集評估（含耗時）
test_start = time.time()
y_pred = adaptive_search.best_estimator_.predict(X_test_scaled)
test_end = time.time()
test_acc = accuracy_score(y_test, y_pred)
print(f"\nSVM測試集準確率: {test_acc:.6f} (預測耗時: {test_end - test_start:.2f} 秒)")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Loading data...
Scaling features...
開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}
迭代 1 最佳參數: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
迭代 1 最佳分數: 0.999693
迭代 1 耗時: 4942.17 秒

迭代 2/3
當前參數搜索範圍: {'C': [500, 666, 832, 998], 'gamma': [0.0001, 0.02005, 0.04, 0.05995], 'kernel': ['rbf']}
迭代 2 最佳參數: {'C': 998, 'gamma': 0.05995, 'kernel': 'rbf'}
迭代 2 最佳分數: 0.999699
迭代 2 耗時: 3372.38 秒

迭代 3/3
當前參數搜索範圍: {'C': [873, 914, 955, 996], 'gamma': [0.044987, 0.049975, 0.054963, 0.05995], 'kernel': ['rbf']}
迭代 3 最佳參數: {'C': 996, 'gamma': 0.049975, 'kernel': 'rbf'}
迭代 3 最佳分數: 0.999702
迭代 3 耗時: 4441.87 秒

自適應參數範圍優化完成!
總耗時: 12756.42 秒
最終最佳參數: {'C': 996, 'gamma': 0.049975, 'kernel': 'rbf'}
最終最佳分數: 0.999702

參數優化歷史:
迭代 1: 分數=0.999693, 參數={'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}, 耗時=4942.17 秒
迭代 2: 分數=0.999699, 參數={'C': 998, 'gamma': 0.05995, 'kernel': 'rbf'}, 耗時=3372.38 秒
迭代 3: 分數=0.999702, 參數={'C': 996, 'gamma': 0.049975, 'kernel':

#  "急速版的自適應(參數收縮幅度較大（shrink_factor=0.25/(iteration+1)）"可以拿來當結果用

每輪只取2~3個參數點

前兩輪只用2折交叉驗證，最後一輪再用5折

計算速度大幅提升，準確率仍然很高)

In [8]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

print("Loading data...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

print("Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class FastAdaptiveGridSearch:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        shrink_factor = 0.25 / (iteration + 1)  # 更大收縮
        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                new_param_grid[param_name] = [param_value]
                continue
            current_values = param_grid[param_name]
            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor
            if param_range < 1e-10:
                new_param_grid[param_name] = current_values
                continue
            lower = max(param_value - delta, min(current_values))
            upper = min(param_value + delta, max(current_values))
            # 只取3個點
            new_values = np.linspace(lower, upper, 3)
            new_param_grid[param_name] = sorted(list(set([round(v, 6) for v in new_values])))
        return new_param_grid

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        print("\n開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            cv_fold = 2 if i < self.n_iterations - 1 else 5  # 前兩輪用2折，最後一輪5折
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            iter_start = time.time()
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=cv_fold, 
                scoring='accuracy',
                n_jobs=-1
            )
            grid_search.fit(X, y)
            iter_end = time.time()
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_end - iter_start:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_end - iter_start
            })
            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}, 耗時={history['iteration_time']:.2f} 秒")
        return self

param_grid = {
    'C': [1, 10, 100, 500, 1000],
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

svm_model = SVC(random_state=42)
adaptive_search = FastAdaptiveGridSearch(
    svm_model, 
    param_grid, 
    n_iterations=3
)

total_train_start = time.time()
adaptive_search.fit(X_train_scaled, y_train)
total_train_end = time.time()
print(f"\nSVM自適應網格搜尋總訓練耗時: {total_train_end - total_train_start:.2f} 秒")

# 測試集評估
test_start = time.time()
y_pred = adaptive_search.best_estimator_.predict(X_test_scaled)
test_end = time.time()
test_acc = accuracy_score(y_test, y_pred)
print(f"\nSVM測試集準確率: {test_acc:.6f} (預測耗時: {test_end - test_start:.2f} 秒)")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Loading data...
Scaling features...

開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}
迭代 1 最佳參數: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
迭代 1 最佳分數: 0.999669
迭代 1 耗時: 2935.02 秒

迭代 2/3
當前參數搜索範圍: {'C': [np.float64(750.25), np.float64(875.125), np.float64(1000.0)], 'gamma': [np.float64(0.0001), np.float64(0.017537), np.float64(0.034975)], 'kernel': ['rbf']}
迭代 2 最佳參數: {'C': np.float64(875.125), 'gamma': np.float64(0.017537), 'kernel': 'rbf'}
迭代 2 最佳分數: 0.999628
迭代 2 耗時: 1179.61 秒

迭代 3/3
當前參數搜索範圍: {'C': [np.float64(843.90625), np.float64(875.125), np.float64(906.34375)], 'gamma': [np.float64(0.013178), np.float64(0.017537), np.float64(0.021896)], 'kernel': ['rbf']}
迭代 3 最佳參數: {'C': np.float64(843.90625), 'gamma': np.float64(0.013178), 'kernel': 'rbf'}
迭代 3 最佳分數: 0.999707
迭代 3 耗時: 1432.86 秒

自適應參數範圍優化完成!
總耗時: 5547.53 秒
最終最佳參數: {'C': np.float64(843.90625), 'gamma': np.float64(0.013178), 'kernel': 'rbf'}
最終最佳分數: 0.999

In [22]:
import pandas as pd
import numpy as np
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
print("載入資料...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
print("特徵標準化...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class FastAdaptiveGridSearch:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        shrink_factor = 0.25 / (iteration + 1)  # 更大收縮
        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                new_param_grid[param_name] = [param_value]
                continue
            current_values = param_grid[param_name]
            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor
            if param_range < 1e-10:
                new_param_grid[param_name] = current_values
                continue
            lower = max(param_value - delta, min(current_values))
            upper = min(param_value + delta, max(current_values))
            # 只取3個點
            new_values = np.linspace(lower, upper, 3)
            new_param_grid[param_name] = sorted(list(set([round(v, 6) for v in new_values])))
        return new_param_grid

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        print("\n開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            cv_fold = 2 if i < self.n_iterations - 1 else 5  # 前兩輪用2折，最後一輪5折
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            iter_start = time.time()
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=cv_fold, 
                scoring='accuracy',
                n_jobs=-1
            )
            grid_search.fit(X, y)
            iter_end = time.time()
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_end - iter_start:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_end - iter_start
            })
            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}, 耗時={history['iteration_time']:.2f} 秒")
        return self

param_grid = {
    'C': [1, 10, 100, 500, 1000],
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

svm_model = SVC(random_state=42)
adaptive_search = FastAdaptiveGridSearch(
    svm_model, 
    param_grid, 
    n_iterations=3
)

total_train_start = time.time()
adaptive_search.fit(X_train_scaled, y_train)
total_train_end = time.time()
print(f"\nSVM自適應網格搜尋總訓練耗時: {total_train_end - total_train_start:.2f} 秒")

# 測試集評估
test_start = time.time()
y_pred = adaptive_search.best_estimator_.predict(X_test_scaled)
test_end = time.time()
test_acc = accuracy_score(y_test, y_pred)
print(f"\nSVM測試集準確率: {test_acc:.6f} (預測耗時: {test_end - test_start:.2f} 秒)")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


載入資料...
特徵標準化...

開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}
迭代 1 最佳參數: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
迭代 1 最佳分數: 0.999669
迭代 1 耗時: 2320.21 秒

迭代 2/3
當前參數搜索範圍: {'C': [np.float64(750.25), np.float64(875.125), np.float64(1000.0)], 'gamma': [np.float64(0.0001), np.float64(0.017537), np.float64(0.034975)], 'kernel': ['rbf']}
迭代 2 最佳參數: {'C': np.float64(875.125), 'gamma': np.float64(0.017537), 'kernel': 'rbf'}
迭代 2 最佳分數: 0.999628
迭代 2 耗時: 590.21 秒

迭代 3/3
當前參數搜索範圍: {'C': [np.float64(843.90625), np.float64(875.125), np.float64(906.34375)], 'gamma': [np.float64(0.013178), np.float64(0.017537), np.float64(0.021896)], 'kernel': ['rbf']}
迭代 3 最佳參數: {'C': np.float64(843.90625), 'gamma': np.float64(0.013178), 'kernel': 'rbf'}
迭代 3 最佳分數: 0.999707
迭代 3 耗時: 842.76 秒

自適應參數範圍優化完成!
總耗時: 3753.20 秒
最終最佳參數: {'C': np.float64(843.90625), 'gamma': np.float64(0.013178), 'kernel': 'rbf'}
最終最佳分數: 0.999707

參數優化歷史:
迭代 1: 分數

## 自適應網格搜尋SVM 測試集 (最終版)

In [7]:
import pandas as pd
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
print("Loading data...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
print("Feature Standardlization...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 改良後的自適應搜尋
class LogRefinedAdaptiveGridSearch:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        for param_name, values in param_grid.items():
            if isinstance(values[0], str):
                new_param_grid[param_name] = values
                continue
            best_val = best_params[param_name]
            log_vals = np.log10(values)
            log_best = np.log10(best_val)
            shrink = 0.7 / (iteration + 1)
            log_range = max(log_vals) - min(log_vals)
            delta = log_range * shrink
            lower = max(log_best - delta, min(log_vals))
            upper = min(log_best + delta, max(log_vals))
            new_logs = np.linspace(lower, upper, 3)
            new_vals = sorted(set([round(float(10**v), 8) for v in new_logs]))
            new_param_grid[param_name] = new_vals
        return new_param_grid

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        print("\nBeginning Adaptive GridSearchCV...")
        for i in range(self.n_iterations):
            cv_fold = 2 if i < self.n_iterations - 1 else 5 
            print(f"\niteration {i+1}/{self.n_iterations}")
            print(f"Current parameter grid: {current_param_grid}")
            iter_start = time.time()
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=cv_fold, 
                scoring='accuracy',
                n_jobs=-1
            )
            grid_search.fit(X, y)
            iter_end = time.time()
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            print(f"iteration {i+1} Best parameter: {iteration_best_params}")
            print(f"iteration {i+1} Best grade: {iteration_best_score:.6f}")
            print(f"iteration {i+1} Execution time: {iter_end - iter_start:.2f} sec")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_end - iter_start
            })
            if iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        elapsed_time = time.time() - start_time
        print("\nAdaptive GridSearchCV completed !")
        print(f"Total execution time: {elapsed_time:.2f} sec")
        print(f"Final best parameter: {self.best_params_}")
        print(f"Final best grade: {self.best_score_:.6f}")
        print("\nThe history of optimized parameter:")
        for history in self.optimization_history:
            print(f"iteration {history['iteration']}: grade={history['best_score']:.6f}, parameter={history['best_params']}, execution time={history['iteration_time']:.2f} sec")
        return self

# 初始化與執行
param_grid = {
    'C': [1, 10, 100, 500, 1000],
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

svm_model = SVC(random_state=42)
adaptive_search = LogRefinedAdaptiveGridSearch(
    svm_model,
    param_grid,
    n_iterations=3
)

total_train_start = time.time()
adaptive_search.fit(X_train_scaled, y_train)
total_train_end = time.time()
print(f"\nSVM adaptive gridsearchCV total execution time: {total_train_end - total_train_start:.2f} sec")

# 測試集評估
test_start = time.time()
y_pred = adaptive_search.best_estimator_.predict(X_test_scaled)
test_end = time.time()
test_acc = accuracy_score(y_test, y_pred)
print(f"\nSVM testing accuracy: {test_acc:.6f} (execution time: {test_end - test_start:.2f} sec)")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Loading data...
Feature Standardlization...

Beginning Adaptive GridSearchCV...

iteration 1/3
Current parameter grid: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}
iteration 1 Best parameter: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
iteration 1 Best grade: 0.999669
iteration 1 Execution time: 1677.54 sec

iteration 2/3
Current parameter grid: {'C': [7.94328235, 89.12509381, 1000.0], 'gamma': [0.0001, 0.00316228, 0.1], 'kernel': ['rbf']}
iteration 2 Best parameter: {'C': 1000.0, 'gamma': 0.00316228, 'kernel': 'rbf'}
iteration 2 Best grade: 0.999634
iteration 2 Execution time: 677.17 sec

iteration 3/3
Current parameter grid: {'C': [184.07720017, 429.04218926, 1000.0], 'gamma': [0.00028184, 0.00316228, 0.03548137], 'kernel': ['rbf']}
iteration 3 Best parameter: {'C': 1000.0, 'gamma': 0.03548137, 'kernel': 'rbf'}
iteration 3 Best grade: 0.999685
iteration 3 Execution time: 1713.70 sec

Adaptive GridSearchCV completed !
Total execution time: 406

In [33]:
import pandas as pd
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
print("載入資料...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
print("特徵標準化...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 改良後的自適應搜尋
class LogRefinedAdaptiveGridSearch:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        shrink_factor = 0.6 / (iteration + 1.5)  # 調整縮減因子，早期更快收斂
        for param_name, values in param_grid.items():
            if isinstance(values[0], str):
                new_param_grid[param_name] = values
                continue
            best_val = best_params[param_name]
            log_vals = np.log10(values)
            log_best = np.log10(best_val)
            log_range = max(log_vals) - min(log_vals)
            delta = log_range * shrink_factor
            lower = max(log_best - delta, min(log_vals))
            upper = min(log_best + delta, max(log_vals))
            new_logs = np.linspace(lower, upper, 4)  # 增加到 4 個點，提升精細度
            new_vals = sorted(set([round(float(10**v), 8) for v in new_logs]))
            new_param_grid[param_name] = new_vals
        return new_param_grid

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        print("\n開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            cv_fold = 3  # 統一使用 3 折交叉驗證
            # print(f"\n迭代 {i+1}/{self.n_iterations} (CV folds: {cv_fold})")
            print(f"當前參數搜索範圍: {current_param_grid}")
            iter_start = time.time()
            grid_search = GridSearchCV(
                self.estimator,
                current_param_grid,
                cv=cv_fold,
                scoring='accuracy',
                n_jobs=-1,
                error_score='raise'  # 提高錯誤敏感度
            )
            grid_search.fit(X, y)
            iter_end = time.time()
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_end - iter_start:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_end - iter_start
            })
            if iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params,
                    current_param_grid,
                    i
                )
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}, 耗時={history['iteration_time']:.2f} 秒")
        return self

# 初始化與執行
param_grid = {
    'C': [1, 10, 100, 500, 1000],  # 保留原始範圍
    'gamma': [0.0001, 0.001, 0.01, 0.1],  # 保留原始範圍
    'kernel': ['rbf']  # 保留原始範圍
}

svm_model = SVC(random_state=42)
adaptive_search = LogRefinedAdaptiveGridSearch(
    svm_model,
    param_grid,
    n_iterations=3  # 保留原始迭代次數
)

total_train_start = time.time()
adaptive_search.fit(X_train_scaled, y_train)
total_train_end = time.time()
print(f"\nSVM自適應網格搜尋總訓練耗時: {total_train_end - total_train_start:.2f} 秒")

# 測試集評估
test_start = time.time()
y_pred = adaptive_search.best_estimator_.predict(X_test_scaled)
test_end = time.time()
test_acc = accuracy_score(y_test, y_pred)
print(f"\nSVM測試集準確率: {test_acc:.6f} (預測耗時: {test_end - test_start:.2f} 秒)")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

載入資料...
特徵標準化...

開始自適應參數範圍優化...
當前參數搜索範圍: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}
迭代 1 最佳參數: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
迭代 1 最佳分數: 0.999685
迭代 1 耗時: 3245.73 秒
當前參數搜索範圍: {'C': [63.09573445, 158.48931925, 398.10717055, 1000.0], 'gamma': [0.00063096, 0.00341455, 0.0184785, 0.1], 'kernel': ['rbf']}
迭代 2 最佳參數: {'C': 1000.0, 'gamma': 0.00341455, 'kernel': 'rbf'}
迭代 2 最佳分數: 0.999688
迭代 2 耗時: 1546.76 秒
當前參數搜索範圍: {'C': [515.22864459, 642.68771732, 801.67806339, 1000.0], 'gamma': [0.00101236, 0.00227685, 0.00512075, 0.01151683], 'kernel': ['rbf']}
迭代 3 最佳參數: {'C': 1000.0, 'gamma': 0.00512075, 'kernel': 'rbf'}
迭代 3 最佳分數: 0.999691
迭代 3 耗時: 600.03 秒

自適應參數範圍優化完成!
總耗時: 5392.52 秒
最終最佳參數: {'C': 1000.0, 'gamma': 0.00512075, 'kernel': 'rbf'}
最終最佳分數: 0.999691

參數優化歷史:
迭代 1: 分數=0.999685, 參數={'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}, 耗時=3245.73 秒
迭代 2: 分數=0.999688, 參數={'C': 1000.0, 'gamma': 0.00341455, 'kernel': 'rbf'}, 耗時=1546.76 秒
迭代 3: 分

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
print("載入資料...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
print("特徵標準化...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class LogRefinedAdaptiveGridSearch:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        for param_name, values in param_grid.items():
            if isinstance(values[0], str):
                new_param_grid[param_name] = values
                continue
            best_val = best_params[param_name]
            log_vals = np.log10(values)
            log_best = np.log10(best_val)
            shrink = 0.7 / (iteration + 1)
            log_range = max(log_vals) - min(log_vals)
            delta = log_range * shrink
            lower = max(log_best - delta, min(log_vals))
            upper = min(log_best + delta, max(log_vals))
            new_logs = np.linspace(lower, upper, 3)
            new_vals = sorted(set([round(float(10**v), 8) for v in new_logs]))
            new_param_grid[param_name] = new_vals
        return new_param_grid

    def _get_cv_folds(self, iteration):
        """根據迭代次數返回對應的CV折數"""
        if iteration == 0:  # 第一次
            return 2
        elif iteration == 1:  # 第二次
            return 4
        else:  # 第三次及以後
            return 5
    

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        print("\n開始自適應參數範圍優化...")
        for i in range(self.n_iterations): 
            cv_fold = self._get_cv_folds(i) 
            
            print(f"\n迭代 {i+1}/{self.n_iterations}") 
            # print(f"CV折數: {cv_fold}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            iter_start = time.time()
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=cv_fold, 
                scoring='accuracy',
                n_jobs=-1
            )
            grid_search.fit(X, y)
            iter_end = time.time()
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_end - iter_start:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_end - iter_start
            })
            if iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}, 耗時={history['iteration_time']:.2f} 秒")
        return self

# 初始化與執行
param_grid = {
    'C': [1, 10, 100, 500, 1000],
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

svm_model = SVC(random_state=42)
adaptive_search = LogRefinedAdaptiveGridSearch(
    svm_model,
    param_grid,
    n_iterations=3
)

total_train_start = time.time()
adaptive_search.fit(X_train_scaled, y_train)
total_train_end = time.time()
print(f"\nSVM自適應網格搜尋總訓練耗時: {total_train_end - total_train_start:.2f} 秒")

# 測試集評估
test_start = time.time()
y_pred = adaptive_search.best_estimator_.predict(X_test_scaled)
test_end = time.time()
test_acc = accuracy_score(y_test, y_pred)
print(f"\nSVM測試集準確率: {test_acc:.6f} (預測耗時: {test_end - test_start:.2f} 秒)")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

載入資料...
特徵標準化...

開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}
迭代 1 最佳參數: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
迭代 1 最佳分數: 0.999669
迭代 1 耗時: 1649.69 秒

迭代 2/3
當前參數搜索範圍: {'C': [7.94328235, 89.12509381, 1000.0], 'gamma': [0.0001, 0.00316228, 0.1], 'kernel': ['rbf']}
迭代 2 最佳參數: {'C': 1000.0, 'gamma': 0.00316228, 'kernel': 'rbf'}
迭代 2 最佳分數: 0.999696
迭代 2 耗時: 2030.90 秒

迭代 3/3
當前參數搜索範圍: {'C': [184.07720017, 429.04218926, 1000.0], 'gamma': [0.00028184, 0.00316228, 0.03548137], 'kernel': ['rbf']}
迭代 3 最佳參數: {'C': 1000.0, 'gamma': 0.03548137, 'kernel': 'rbf'}
迭代 3 最佳分數: 0.999685
迭代 3 耗時: 1648.94 秒

自適應參數範圍優化完成!
總耗時: 5329.54 秒
最終最佳參數: {'C': 1000.0, 'gamma': 0.00316228, 'kernel': 'rbf'}
最終最佳分數: 0.999696

參數優化歷史:
迭代 1: 分數=0.999669, 參數={'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}, 耗時=1649.69 秒
迭代 2: 分數=0.999696, 參數={'C': 1000.0, 'gamma': 0.00316228, 'kernel': 'rbf'}, 耗時=2030.90 秒
迭代 3: 分數=0.999685, 參數={'C': 1000.0, 'ga

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
print("載入資料...")
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
print("特徵標準化...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class LogRefinedAdaptiveGridSearch:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        for param_name, values in param_grid.items():
            if isinstance(values[0], str):
                new_param_grid[param_name] = values
                continue
            best_val = best_params[param_name]
            log_vals = np.log10(values)
            log_best = np.log10(best_val)
            shrink = 0.7 / (iteration + 1)
            log_range = max(log_vals) - min(log_vals)
            delta = log_range * shrink
            lower = max(log_best - delta, min(log_vals))
            upper = min(log_best + delta, max(log_vals))
            new_logs = np.linspace(lower, upper, 3)
            new_vals = sorted(set([round(float(10**v), 8) for v in new_logs]))
            new_param_grid[param_name] = new_vals
        return new_param_grid

    def fit(self, X, y):
        start_time = time.time()
        current_param_grid = self.initial_param_grid
        print("\n開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            # 修改這裡：第一次使用 cv=2，之後使用 cv=5
            cv_fold = 2 if i == 0 else 5

            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")
            iter_start = time.time()
            grid_search = GridSearchCV(
                self.estimator, 
                current_param_grid, 
                cv=cv_fold, 
                scoring='accuracy',
                n_jobs=-1
            )
            grid_search.fit(X, y)
            iter_end = time.time()
            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_end - iter_start:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_end - iter_start
            })
            if iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = grid_search.best_estimator_
            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(
                    iteration_best_params, 
                    current_param_grid,
                    i
                )
        elapsed_time = time.time() - start_time
        print("\n自適應參數範圍優化完成!")
        print(f"總耗時: {elapsed_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}, 耗時={history['iteration_time']:.2f} 秒")
        return self

# 初始化與執行
param_grid = {
    'C': [1, 10, 100, 500, 1000],
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf']
}

svm_model = SVC(random_state=42)
adaptive_search = LogRefinedAdaptiveGridSearch(
    svm_model,
    param_grid,
    n_iterations=3
)

total_train_start = time.time()
adaptive_search.fit(X_train_scaled, y_train)
total_train_end = time.time()
print(f"\nSVM自適應網格搜尋總訓練耗時: {total_train_end - total_train_start:.2f} 秒")

# 測試集評估
test_start = time.time()
y_pred = adaptive_search.best_estimator_.predict(X_test_scaled)
test_end = time.time()
test_acc = accuracy_score(y_test, y_pred)
print(f"\nSVM測試集準確率: {test_acc:.6f} (預測耗時: {test_end - test_start:.2f} 秒)")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

載入資料...
特徵標準化...

開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [1, 10, 100, 500, 1000], 'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['rbf']}
迭代 1 最佳參數: {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
迭代 1 最佳分數: 0.999669
迭代 1 耗時: 1721.89 秒

迭代 2/3
當前參數搜索範圍: {'C': [7.94328235, 89.12509381, 1000.0], 'gamma': [0.0001, 0.00316228, 0.1], 'kernel': ['rbf']}
迭代 2 最佳參數: {'C': 1000.0, 'gamma': 0.00316228, 'kernel': 'rbf'}
迭代 2 最佳分數: 0.999680
迭代 2 耗時: 2378.04 秒

迭代 3/3
當前參數搜索範圍: {'C': [184.07720017, 429.04218926, 1000.0], 'gamma': [0.00028184, 0.00316228, 0.03548137], 'kernel': ['rbf']}
迭代 3 最佳參數: {'C': 1000.0, 'gamma': 0.03548137, 'kernel': 'rbf'}
迭代 3 最佳分數: 0.999685
迭代 3 耗時: 1701.91 秒

自適應參數範圍優化完成!
總耗時: 5801.85 秒
最終最佳參數: {'C': 1000.0, 'gamma': 0.03548137, 'kernel': 'rbf'}
最終最佳分數: 0.999685

參數優化歷史:
迭代 1: 分數=0.999669, 參數={'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}, 耗時=1721.89 秒
迭代 2: 分數=0.999680, 參數={'C': 1000.0, 'gamma': 0.00316228, 'kernel': 'rbf'}, 耗時=2378.04 秒
迭代 3: 分數=0.999685, 參數={'C': 1000.0, 'ga

# "用tesing data(之前模型都沒有看過的測試資料(資料夾03-11)來個別測試過自適應往格搜尋調整過後的LR模型的測試準確率(模型最終的表現)"

In [31]:
import os
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import time

# 添加自適應參數範圍優化類
class AdaptiveGridSearch:
    def __init__(self, estimator, param_grid, cv=5, scoring='accuracy', n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_score_ = 0
        self.best_estimator_ = None
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        shrink_factor = 0.3 / (iteration + 1)

        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                new_param_grid[param_name] = [param_value]
                continue

            current_values = param_grid[param_name]
            if len(current_values) <= 1:
                new_param_grid[param_name] = current_values
                continue

            try:
                idx = current_values.index(param_value)
            except ValueError:
                idx = min(range(len(current_values)), key=lambda i: abs(current_values[i] - param_value))

            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor

            if param_range < 1e-10:
                new_param_grid[param_name] = current_values
                continue

            if isinstance(param_value, float):
                log_min = np.log10(min(current_values))
                log_max = np.log10(max(current_values))
                log_val = np.log10(param_value)
                log_delta = (log_max - log_min) * shrink_factor
                log_lower = max(log_val - log_delta, log_min)
                log_upper = min(log_val + log_delta, log_max)

                new_values = [round(10**x, 6) for x in np.linspace(log_lower, log_upper, 8)]

            else:
                log_min = np.log10(min(current_values))
                log_max = np.log10(max(current_values))
                log_val = np.log10(param_value)
                log_delta = (log_max - log_min) * shrink_factor
                log_lower = max(log_val - log_delta, log_min)
                log_upper = min(log_val + log_delta, log_max)

                new_values = [round(10 ** x, 6) for x in np.linspace(log_lower, log_upper, 4)]

            new_param_grid[param_name] = sorted(list(set(new_values)))

        return new_param_grid

    def fit(self, X, y, X_test=None, y_test=None):
        start_time = time.time()
        current_param_grid = self.initial_param_grid

        print("開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")

            grid_search = GridSearchCV(
                self.estimator,
                current_param_grid,
                cv=self.cv,
                scoring=self.scoring,
                n_jobs=-1
            )
            grid_search.fit(X, y)

            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            iteration_best_estimator = grid_search.best_estimator_

            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")

            # 額外輸出測試集評估
            if X_test is not None and y_test is not None:
                y_pred = iteration_best_estimator.predict(X_test)
                test_acc = accuracy_score(y_test, y_pred)
                print(f"迭代 {i+1} 測試集準確率: {test_acc:.6f}")

            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score
            })

            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = iteration_best_estimator

            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(iteration_best_params, current_param_grid, i)

        print("\n自適應參數範圍優化完成!")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        return self

# 載入資料
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵縮放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression 的自適應參數優化
param_grid_lr = {
    'C': [1e-5, 1e-4, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}
logreg_model = LogisticRegression(max_iter=1000, random_state=42)
adaptive_search_lr = AdaptiveGridSearch(logreg_model, param_grid_lr, cv=5, scoring='accuracy', n_iterations=3)
adaptive_search_lr.fit(X_train_scaled, y_train, X_test_scaled, y_test)
best_lr_model = adaptive_search_lr.best_estimator_

# 模型測試集評估
def evaluate_model(model, X_test, y_test, model_name="Model"):
    y_pred_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_test)
    print(f"{model_name} 測試準確率: {accuracy:.6f}")
    print(f"{model_name} 分類報告:\n{classification_report(y_test, y_pred_test)}")
    print(f"{model_name} 混淆矩陣:\n{confusion_matrix(y_test, y_pred_test)}")

print("\n===== 測試集評估：Logistic Regression (Adaptive) =====")
evaluate_model(best_lr_model, X_test_scaled, y_test, "Logistic Regression (Adaptive)")


開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l2'], 'solver': ['lbfgs']}
迭代 1 最佳參數: {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
迭代 1 最佳分數: 0.990380
迭代 1 測試集準確率: 0.996830

迭代 2/3
當前參數搜索範圍: {'C': [np.float64(1e-05), np.float64(4.3e-05), np.float64(0.000181), np.float64(0.000769), np.float64(0.003268), np.float64(0.013895), np.float64(0.059078), np.float64(0.251189)], 'penalty': ['l2'], 'solver': ['lbfgs']}
迭代 2 最佳參數: {'C': np.float64(0.003268), 'penalty': 'l2', 'solver': 'lbfgs'}
迭代 2 最佳分數: 0.990632
迭代 2 測試集準確率: 0.996776

迭代 3/3
當前參數搜索範圍: {'C': [np.float64(0.000715), np.float64(0.001104), np.float64(0.001704), np.float64(0.00263), np.float64(0.00406), np.float64(0.006268), np.float64(0.009676), np.float64(0.014938)], 'penalty': ['l2'], 'solver': ['lbfgs']}
迭代 3 最佳參數: {'C': np.float64(0.00406), 'penalty': 'l2', 'solver': 'lbfgs'}
迭代 3 最佳分數: 0.990646
迭代 3 測試集準確率: 0.996803

自適應參數範圍優化完成!
最終最佳參數: {'C': np.float64(0.00406), 'penal

## 最終LR自適應結果

In [4]:
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class FastAdaptiveGridSearch:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []
        self.allowed_C = param_grid['C']

    def _get_neighbor_C(self, best_C):
        idx = self.allowed_C.index(best_C)
        neighbors = []
        if idx > 0:
            neighbors.append(self.allowed_C[idx-1])
        neighbors.append(best_C)
        if idx < len(self.allowed_C)-1:
            neighbors.append(self.allowed_C[idx+1])
        return sorted(list(set(neighbors)))

    def _generate_new_param_grid(self, best_params):
        new_grid = {}
        # 只取最佳C的左右各一個
        if 'C' in best_params:
            best_C = best_params['C']
            new_grid['C'] = self._get_neighbor_C(best_C)
        for param in ['penalty', 'solver']:
            if param in best_params:
                new_grid[param] = [best_params[param]]
        return new_grid

    def fit(self, X, y):
        start_time = time.time()
        current_grid = self.initial_param_grid
        print("begin LR model adaptive gridsearchCV...")
        for i in range(self.n_iterations):
            cv = 2 if i < self.n_iterations - 1 else 5  # 前兩輪2折，最後一輪5折
            print(f"\niteration {i+1}/{self.n_iterations}")
            print(f"current grid: {current_grid}")
            gs = GridSearchCV(
                self.estimator, 
                current_grid, 
                cv=cv, 
                scoring='accuracy',
                n_jobs=-1,
                error_score='raise'
            )
            iter_start = time.time()
            gs.fit(X, y)
            iter_time = time.time() - iter_start
            self.best_params_ = gs.best_params_
            self.best_estimator_ = gs.best_estimator_
            self.best_score_ = gs.best_score_
            print(f"iteration {i+1} best parameter: {self.best_params_}")
            print(f"iteration {i+1} best grade: {self.best_score_:.6f}")
            print(f"iteration {i+1} execution time: {iter_time:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_grid,
                'best_params': self.best_params_,
                'best_score': self.best_score_,
                'iteration_time': iter_time
            })
            if i < self.n_iterations - 1:
                current_grid = self._generate_new_param_grid(self.best_params_)
        total_time = time.time() - start_time
        print(f"\ntotal execution time: {total_time:.2f} 秒")
        print(f"final best parameter: {self.best_params_}")
        print(f"final best grade: {self.best_score_:.6f}")
        print("\nthe history of optimized parameter:")
        for history in self.optimization_history:
            print(f"iteration {history['iteration']}: grade={history['best_score']:.6f}, parameter={history['best_params']}, execution time={history['iteration_time']:.2f} sec")
        return self

# 與GridSearch相同的初始參數範圍
param_grid_lr = {
    'C': [1e-5, 1e-4, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}

logreg_model = LogisticRegression(max_iter=1000, random_state=42)
fast_searcher = FastAdaptiveGridSearch(logreg_model, param_grid_lr, n_iterations=3)
fast_searcher.fit(X_train_scaled, y_train)

# 測試集評估
def evaluate_model(model, X_test, y_test, model_name="Model"):
    start = time.time()
    y_pred = model.predict(X_test)
    infer_time = time.time() - start
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{model_name} testing accuracy: {acc:.6f}")
    print(f"classification_report:\n{classification_report(y_test, y_pred)}")
    print(f"confusion_matrix:\n{confusion_matrix(y_test, y_pred)}")

evaluate_model(fast_searcher.best_estimator_, X_test_scaled, y_test, "Logistic Regression (Adaptive)")


begin LR model adaptive gridsearchCV...

iteration 1/3
current grid: {'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l2'], 'solver': ['lbfgs']}
iteration 1 best parameter: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
iteration 1 best grade: 0.990543
iteration 1 execution time: 6.84 秒

iteration 2/3
current grid: {'C': [0.001, 0.01, 0.1], 'penalty': ['l2'], 'solver': ['lbfgs']}
iteration 2 best parameter: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
iteration 2 best grade: 0.990543
iteration 2 execution time: 3.50 秒

iteration 3/3
current grid: {'C': [0.001, 0.01, 0.1], 'penalty': ['l2'], 'solver': ['lbfgs']}
iteration 3 best parameter: {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
iteration 3 best grade: 0.990380
iteration 3 execution time: 5.83 秒

total execution time: 16.16 秒
final best parameter: {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}
final best grade: 0.990380

the history of optimized parameter:
iteration 1: grade=0.990543, parameter={'C'

# "用tesing data(之前模型都沒有看過的測試資料(資料夾03-11)來個別測試過自適應往格搜尋調整過後的KNN模型的測試準確率(模型最終的表現)"

In [1]:
import os
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# 自適應網格搜尋器（與你提供的版本一致）
class AdaptiveGridSearch:
    def __init__(self, estimator, param_grid, cv=5, scoring='accuracy', n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_score_ = 0
        self.best_estimator_ = None
        self.optimization_history = []

    def _generate_new_param_grid(self, best_params, param_grid, iteration):
        new_param_grid = {}
        shrink_factor = 0.5 / (iteration + 1)

        for param_name, param_value in best_params.items():
            if param_name not in param_grid or isinstance(param_grid[param_name][0], str):
                new_param_grid[param_name] = [param_value]
                continue

            current_values = param_grid[param_name]
            if len(current_values) <= 1:
                new_param_grid[param_name] = current_values
                continue

            try:
                idx = current_values.index(param_value)
            except ValueError:
                idx = min(range(len(current_values)), key=lambda i: abs(current_values[i] - param_value))

            param_range = max(current_values) - min(current_values)
            delta = param_range * shrink_factor

            if param_range < 1e-10:
                new_param_grid[param_name] = current_values
                continue

            if isinstance(param_value, int):
                lower = max(int(param_value - delta), min(current_values))
                upper = min(int(param_value + delta), max(current_values))
                if lower == upper:
                    new_values = [lower]
                else:
                    step = max(1, (upper - lower) // 3)
                    new_values = list(range(lower, upper + 1, step))
            else:
                lower = max(param_value - delta, min(current_values))
                upper = min(param_value + delta, max(current_values))
                step = (upper - lower) / 3
                new_values = [round(lower + i * step, 6) for i in range(4)]

            new_param_grid[param_name] = sorted(list(set(new_values)))

        return new_param_grid

    def fit(self, X, y, X_test=None, y_test=None):
        start_time = time.time()
        current_param_grid = self.initial_param_grid

        print("開始自適應參數範圍優化...")
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數搜索範圍: {current_param_grid}")

            grid_search = GridSearchCV(
                self.estimator,
                current_param_grid,
                cv=self.cv,
                scoring=self.scoring,
                n_jobs=-1
            )
            grid_search.fit(X, y)

            iteration_best_params = grid_search.best_params_
            iteration_best_score = grid_search.best_score_
            iteration_best_estimator = grid_search.best_estimator_

            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")

            if X_test is not None and y_test is not None:
                y_pred = iteration_best_estimator.predict(X_test)
                test_acc = accuracy_score(y_test, y_pred)
                print(f"迭代 {i+1} 測試集準確率: {test_acc:.6f}")

            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_param_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score
            })

            if i == 0 or iteration_best_score > self.best_score_:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = iteration_best_estimator

            if i < self.n_iterations - 1:
                current_param_grid = self._generate_new_param_grid(iteration_best_params, current_param_grid, i)

        print("\n自適應參數範圍優化完成!")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        return self

# 載入資料
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵縮放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 自適應 KNN 模型設定與訓練
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1: 曼哈頓距離, 2: 歐幾里得距離
}
knn_model = KNeighborsClassifier()
adaptive_search_knn = AdaptiveGridSearch(knn_model, param_grid_knn, cv=5, scoring='accuracy', n_iterations=3)
adaptive_search_knn.fit(X_train_scaled, y_train, X_test_scaled, y_test)
best_knn_model = adaptive_search_knn.best_estimator_

# 測試集模型評估函式
def evaluate_model(model, X_test, y_test, model_name="Model"):
    y_pred_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_test)
    print(f"{model_name} 測試準確率: {accuracy:.6f}")
    print(f"{model_name} 分類報告:\n{classification_report(y_test, y_pred_test)}")
    print(f"{model_name} 混淆矩陣:\n{confusion_matrix(y_test, y_pred_test)}")

# 模型測試集評估
print("\n===== 測試集評估：KNN (Adaptive) =====")
evaluate_model(best_knn_model, X_test_scaled, y_test, "KNN (Adaptive)")


開始自適應參數範圍優化...

迭代 1/3
當前參數搜索範圍: {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'p': [1, 2]}
迭代 1 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 1 最佳分數: 0.999799
迭代 1 測試集準確率: 0.998395

迭代 2/3
當前參數搜索範圍: {'n_neighbors': [3, 4, 5, 6, 7], 'p': [1], 'weights': ['distance']}
迭代 2 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 2 最佳分數: 0.999799
迭代 2 測試集準確率: 0.998395

迭代 3/3
當前參數搜索範圍: {'n_neighbors': [3, 4], 'p': [1], 'weights': ['distance']}
迭代 3 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 3 最佳分數: 0.999799
迭代 3 測試集準確率: 0.998395

自適應參數範圍優化完成!
最終最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
最終最佳分數: 0.999799

===== 測試集評估：KNN (Adaptive) =====
KNN (Adaptive) 測試準確率: 0.998395
KNN (Adaptive) 分類報告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49763
           1       1.00      1.00      1.00     25620

    accuracy                           1.00     75383
   macro avg       

In [11]:
import pandas as pd
import numpy as np
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class FastAdaptiveKNN:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []
        self.allowed_k = sorted(param_grid['n_neighbors'])

    def _get_neighbor_k(self, best_k):
        idx = self.allowed_k.index(best_k)
        neighbors = []
        if idx > 0:
            neighbors.append(self.allowed_k[idx-1])
        neighbors.append(best_k)
        if idx < len(self.allowed_k)-1:
            neighbors.append(self.allowed_k[idx+1])
        return sorted(list(set(neighbors)))

    def _generate_new_param_grid(self, best_params):
        new_grid = {}
        # 只取最佳k的左右鄰居
        if 'n_neighbors' in best_params:
            best_k = best_params['n_neighbors']
            new_grid['n_neighbors'] = self._get_neighbor_k(best_k)
        # weights和p維持最佳
        for param in ['weights', 'p']:
            if param in best_params:
                new_grid[param] = [best_params[param]]
        return new_grid

    def fit(self, X, y):
        start_time = time.time()
        current_grid = self.initial_param_grid
        print("開始急速版KNN參數搜索...")
        for i in range(self.n_iterations):
            cv = 2 if i < self.n_iterations - 1 else 5
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數網格: {current_grid}")
            gs = GridSearchCV(
                self.estimator, 
                current_grid, 
                cv=cv, 
                scoring='accuracy',
                n_jobs=-1,
                error_score='raise'
            )
            iter_start = time.time()
            gs.fit(X, y)
            iter_time = time.time() - iter_start
            self.best_params_ = gs.best_params_
            self.best_estimator_ = gs.best_estimator_
            self.best_score_ = gs.best_score_
            print(f"迭代 {i+1} 最佳參數: {self.best_params_}")
            print(f"迭代 {i+1} 最佳分數: {self.best_score_:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_time:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_grid,
                'best_params': self.best_params_,
                'best_score': self.best_score_,
                'iteration_time': iter_time
            })
            if i < self.n_iterations - 1:
                current_grid = self._generate_new_param_grid(self.best_params_)
        total_time = time.time() - start_time
        print(f"\n總耗時: {total_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        print("\n參數優化歷史:")
        for history in self.optimization_history:
            print(f"迭代 {history['iteration']}: 分數={history['best_score']:.6f}, 參數={history['best_params']}, 耗時={history['iteration_time']:.2f} 秒")
        return self

# 初始參數範圍與你GridSearch相同
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

knn_model = KNeighborsClassifier()
fast_knn_search = FastAdaptiveKNN(knn_model, param_grid_knn, n_iterations=3)
fast_knn_search.fit(X_train_scaled, y_train)

# 測試集評估
def evaluate_model(model, X_test, y_test, model_name="Model"):
    start = time.time()
    y_pred = model.predict(X_test)
    infer_time = time.time() - start
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{model_name} 測試準確率: {acc:.6f}")
    print(f"推理耗時: {infer_time:.2f} 秒")
    print(f"分類報告:\n{classification_report(y_test, y_pred)}")
    print(f"混淆矩陣:\n{confusion_matrix(y_test, y_pred)}")

evaluate_model(fast_knn_search.best_estimator_, X_test_scaled, y_test, "KNN (急速版)")


開始急速版KNN參數搜索...

迭代 1/3
當前參數網格: {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'p': [1, 2]}
迭代 1 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 1 最佳分數: 0.999704
迭代 1 耗時: 3172.29 秒

迭代 2/3
當前參數網格: {'n_neighbors': [3, 5], 'weights': ['distance'], 'p': [1]}
迭代 2 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 2 最佳分數: 0.999704
迭代 2 耗時: 2023.16 秒

迭代 3/3
當前參數網格: {'n_neighbors': [3, 5], 'weights': ['distance'], 'p': [1]}
迭代 3 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 3 最佳分數: 0.999799
迭代 3 耗時: 1311.95 秒

總耗時: 6507.43 秒
最終最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
最終最佳分數: 0.999799

參數優化歷史:
迭代 1: 分數=0.999704, 參數={'n_neighbors': 3, 'p': 1, 'weights': 'distance'}, 耗時=3172.29 秒
迭代 2: 分數=0.999704, 參數={'n_neighbors': 3, 'p': 1, 'weights': 'distance'}, 耗時=2023.16 秒
迭代 3: 分數=0.999799, 參數={'n_neighbors': 3, 'p': 1, 'weights': 'distance'}, 耗時=1311.95 秒

KNN (急速版) 測試準確率: 0.998395
推理耗時: 122.87 秒
分類報告:
              precision    re

## 最終版KNN自適應結果

In [13]:
import pandas as pd
import numpy as np
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class RefinedAdaptiveKNN:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []
        self.allowed_k = sorted(param_grid['n_neighbors'])

    def _get_refined_k_range(self, best_k):
        min_k = max(min(self.allowed_k), best_k - 2)
        max_k = min(max(self.allowed_k), best_k + 2)
        return sorted(list(set([k for k in range(min_k, max_k + 1)]).intersection(self.allowed_k)))

    def _generate_new_param_grid(self, best_params):
        new_grid = {}
        if 'n_neighbors' in best_params:
            best_k = best_params['n_neighbors']
            new_grid['n_neighbors'] = self._get_refined_k_range(best_k)
        for param in ['weights', 'p']:
            if param in best_params:
                new_grid[param] = [best_params[param]]
        return new_grid

    def fit(self, X, y):
        start_time = time.time()
        current_grid = self.initial_param_grid
        print("開始自適應版KNN參數搜索...")
        for i in range(self.n_iterations):
            cv = 2 if i < self.n_iterations - 1 else 5
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數網格: {current_grid}")
            gs = GridSearchCV(
                self.estimator, 
                current_grid, 
                cv=cv, 
                scoring='accuracy',
                n_jobs=-1,
                error_score='raise'
            )
            iter_start = time.time()
            gs.fit(X, y)
            iter_time = time.time() - iter_start
            self.best_params_ = gs.best_params_
            self.best_estimator_ = gs.best_estimator_
            self.best_score_ = gs.best_score_
            print(f"迭代 {i+1} 最佳參數: {self.best_params_}")
            print(f"迭代 {i+1} 最佳分數: {self.best_score_:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_time:.2f} 秒")
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_grid,
                'best_params': self.best_params_,
                'best_score': self.best_score_,
                'iteration_time': iter_time
            })
            if i < self.n_iterations - 1:
                current_grid = self._generate_new_param_grid(self.best_params_)
        total_time = time.time() - start_time
        print(f"\n總耗時: {total_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        return self, total_time

# 初始參數範圍與一般 GridSearch 相同
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

knn_model = KNeighborsClassifier()
refined_knn_search = RefinedAdaptiveKNN(knn_model, param_grid_knn, n_iterations=3)
refined_knn_search, total_duration = refined_knn_search.fit(X_train_scaled, y_train)

# 測試集評估
def evaluate_model(model, X_test, y_test, model_name="Model"):
    start = time.time()
    y_pred = model.predict(X_test)
    infer_time = time.time() - start
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{model_name} 測試準確率: {acc:.6f}")
    print(f"推理耗時: {infer_time:.2f} 秒")
    print(f"分類報告:\n{classification_report(y_test, y_pred)}")
    print(f"混淆矩陣:\n{confusion_matrix(y_test, y_pred)}")
    return acc

evaluate_model(refined_knn_search.best_estimator_, X_test_scaled, y_test, "KNN (自適應版)")
print(f"\n總搜尋耗時: {total_duration:.2f} 秒")


開始自適應版KNN參數搜索...

迭代 1/3
當前參數網格: {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'p': [1, 2]}
迭代 1 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 1 最佳分數: 0.999704
迭代 1 耗時: 3128.36 秒

迭代 2/3
當前參數網格: {'n_neighbors': [3, 5], 'weights': ['distance'], 'p': [1]}
迭代 2 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 2 最佳分數: 0.999704
迭代 2 耗時: 2027.26 秒

迭代 3/3
當前參數網格: {'n_neighbors': [3, 5], 'weights': ['distance'], 'p': [1]}
迭代 3 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 3 最佳分數: 0.999799
迭代 3 耗時: 1299.58 秒

總耗時: 6455.22 秒
最終最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
最終最佳分數: 0.999799

KNN (自適應版) 測試準確率: 0.998395
推理耗時: 118.95 秒
分類報告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49763
           1       1.00      1.00      1.00     25620

    accuracy                           1.00     75383
   macro avg       1.00      1.00      1.00     75383
weighted avg       1.0

## 自適應網格搜尋的KNN 測試集(最終版)

In [20]:
import pandas as pd
import numpy as np
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class OptimizedRefinedAdaptiveKNN:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []
        self.allowed_k = sorted(param_grid['n_neighbors'])
        
        # 預先計算用於抽樣的索引 - 針對大型數據集進行優化
        self.sample_size = min(50000, len(X_train_scaled))  # 限制樣本大小以加速搜索
        
    def _get_refined_k_range(self, best_k, iteration):
        """動態調整k的搜索範圍，隨迭代變得更精細"""
        if iteration == 0:
            # 第一次迭代：保持原始範圍
            return self.allowed_k
        elif iteration == 1:
            # 第二次迭代：圍繞最佳k值±2
            min_k = max(min(self.allowed_k), best_k - 2)
            max_k = min(max(self.allowed_k), best_k + 2)
            refined_range = [k for k in self.allowed_k if min_k <= k <= max_k]
            return refined_range if refined_range else [best_k]
        else:
            # 第三次迭代：圍繞最佳k值±1，更精細搜索
            min_k = max(min(self.allowed_k), best_k - 1)
            max_k = min(max(self.allowed_k), best_k + 1)
            refined_range = [k for k in self.allowed_k if min_k <= k <= max_k]
            return refined_range if refined_range else [best_k]

    def _generate_new_param_grid(self, best_params, iteration):
        """根據迭代次數生成不同精細度的參數網格"""
        new_grid = {}
        
        # k值範圍調整
        if 'n_neighbors' in best_params:
            best_k = best_params['n_neighbors']
            new_grid['n_neighbors'] = self._get_refined_k_range(best_k, iteration)
        
        # weights和p參數的策略調整
        if iteration == 0:
            # 第一次迭代：保持所有選項
            for param in ['weights', 'p']:
                if param in self.initial_param_grid:
                    new_grid[param] = self.initial_param_grid[param]
        elif iteration == 1:
            # 第二次迭代：基於第一次的結果，但仍保持一定探索性
            if 'weights' in best_params:
                # 保留最佳weights，但也保留另一個選項進行比較
                best_weight = best_params['weights']
                other_weights = [w for w in self.initial_param_grid['weights'] if w != best_weight]
                new_grid['weights'] = [best_weight] + other_weights[:1]  # 最多保留2個選項
            
            if 'p' in best_params:
                # 對p參數採用類似策略
                best_p = best_params['p']
                other_p = [p for p in self.initial_param_grid['p'] if p != best_p]
                new_grid['p'] = [best_p] + other_p[:1]  # 最多保留2個選項
        else:
            # 第三次迭代：專注於最佳參數組合的精細調優
            for param in ['weights', 'p']:
                if param in best_params:
                    new_grid[param] = [best_params[param]]
        
        return new_grid

    def _get_sample_indices(self, y, sample_size):
        """分層抽樣獲取訓練子集，確保類別平衡"""
        if len(y) <= sample_size:
            return np.arange(len(y))
        
        # 使用分層抽樣保持類別比例
        from sklearn.model_selection import train_test_split
        indices = np.arange(len(y))
        sample_indices, _ = train_test_split(
            indices, 
            train_size=sample_size, 
            stratify=y, 
            random_state=42
        )
        return sample_indices

    def fit(self, X, y):
        start_time = time.time()
        current_grid = self.initial_param_grid
        
        print("開始自適應KNN參數搜索...")
        # print(f"數據集大小: {X.shape}")
        
        # 預先獲取抽樣索引用於前兩次迭代
        sample_indices = self._get_sample_indices(y, self.sample_size)
        # print(f"前兩次迭代使用樣本大小: {len(sample_indices)}")
        
        for i in range(self.n_iterations):
            print(f"\n迭代 {i+1}/{self.n_iterations}")
            print(f"當前參數網格: {current_grid}")
            
            # 動態決定使用的數據量和交叉驗證策略
            if i < self.n_iterations - 1:
                # 前兩次迭代：使用抽樣數據和較少的CV折數
                X_iter = X[sample_indices]
                y_iter = y.iloc[sample_indices] if hasattr(y, 'iloc') else y[sample_indices]
                cv_folds = 3  # 使用3折交叉驗證加速
                # print(f"使用抽樣數據: {X_iter.shape[0]} 樣本, CV折數: {cv_folds}")
            else:
                # 最後一次迭代：使用完整數據和標準CV折數
                X_iter = X
                y_iter = y
                cv_folds = 5  # 最後一次使用完整5折交叉驗證確保準確性
                # print(f"使用完整數據: {X_iter.shape[0]} 樣本, CV折數: {cv_folds}")
            
            # 使用StratifiedKFold確保類別平衡
            skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
            
            gs = GridSearchCV(
                self.estimator, 
                current_grid, 
                cv=skf,
                scoring='accuracy',
                n_jobs=-1,
                error_score='raise',
                verbose=0  # 減少輸出
            )
            
            iter_start = time.time()
            gs.fit(X_iter, y_iter)
            iter_time = time.time() - iter_start
            
            # 更新最佳參數
            iteration_best_params = gs.best_params_
            iteration_best_score = gs.best_score_
            
            # 如果是最後一次迭代或者分數有提升，更新全局最佳
            if i == 0 or iteration_best_score > self.best_score_ or i == self.n_iterations - 1:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                # 使用完整數據訓練最終模型
                self.best_estimator_ = self.estimator.set_params(**self.best_params_)
                if i < self.n_iterations - 1:
                    # 如果不是最後一次迭代，用完整數據重新訓練以獲得更準確的模型
                    self.best_estimator_.fit(X, y)
                else:
                    # 最後一次迭代已經使用完整數據
                    self.best_estimator_ = gs.best_estimator_
            
            print(f"迭代 {i+1} 最佳參數: {iteration_best_params}")
            print(f"迭代 {i+1} 最佳分數: {iteration_best_score:.6f}")
            print(f"迭代 {i+1} 耗時: {iter_time:.2f} 秒")
            
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_time,
                'sample_size': len(y_iter)
            })
            
            # 為下一次迭代生成新的參數網格
            if i < self.n_iterations - 1:
                current_grid = self._generate_new_param_grid(iteration_best_params, i)
        
        total_time = time.time() - start_time
        print(f"\n總耗時: {total_time:.2f} 秒")
        print(f"最終最佳參數: {self.best_params_}")
        print(f"最終最佳分數: {self.best_score_:.6f}")
        
        # 輸出優化歷史摘要
        print(f"\n優化歷史摘要:")
        for hist in self.optimization_history:
            print(f"迭代{hist['iteration']}: 樣本數={hist['sample_size']}, "
                  f"參數數量={np.prod([len(v) for v in hist['param_grid'].values()])}, "
                  f"分數={hist['best_score']:.6f}, 耗時={hist['iteration_time']:.2f}s")
        
        return self, total_time

# 初始參數範圍與一般 GridSearch 相同（保持不變）
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

knn_model = KNeighborsClassifier()
optimized_knn_search = OptimizedRefinedAdaptiveKNN(knn_model, param_grid_knn, n_iterations=3)
optimized_knn_search, total_duration = optimized_knn_search.fit(X_train_scaled, y_train)

# 測試集評估
def evaluate_model(model, X_test, y_test, model_name="Model"):
    start = time.time()
    y_pred = model.predict(X_test)
    infer_time = time.time() - start
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{model_name} 測試準確率: {acc:.6f}")
    print(f"推理耗時: {infer_time:.2f} 秒")
    print(f"分類報告:\n{classification_report(y_test, y_pred)}")
    print(f"混淆矩陣:\n{confusion_matrix(y_test, y_pred)}")
    return acc

final_accuracy = evaluate_model(optimized_knn_search.best_estimator_, X_test_scaled, y_test, "KNN (自適應)")
print(f"\n總搜尋耗時: {total_duration:.2f} 秒")
# print(f"與一般網格搜索({5130.49:.2f}秒)相比，速度提升: {5130.49/total_duration:.1f}倍")
# print(f"準確率比較 - 目標: 0.998395, 實際: {final_accuracy:.6f}, 差異: {abs(final_accuracy-0.998395)*100:.4f}%")

開始自適應KNN參數搜索...

迭代 1/3
當前參數網格: {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'p': [1, 2]}
迭代 1 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 1 最佳分數: 0.999140
迭代 1 耗時: 88.77 秒

迭代 2/3
當前參數網格: {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'p': [1, 2]}
迭代 2 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 2 最佳分數: 0.999140
迭代 2 耗時: 84.65 秒

迭代 3/3
當前參數網格: {'n_neighbors': [3, 5], 'weights': ['distance', 'uniform'], 'p': [1, 2]}
迭代 3 最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
迭代 3 最佳分數: 0.999797
迭代 3 耗時: 2039.72 秒

總耗時: 2213.50 秒
最終最佳參數: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
最終最佳分數: 0.999797

優化歷史摘要:
迭代1: 樣本數=50000, 參數數量=20, 分數=0.999140, 耗時=88.77s
迭代2: 樣本數=50000, 參數數量=20, 分數=0.999140, 耗時=84.65s
迭代3: 樣本數=368604, 參數數量=8, 分數=0.999797, 耗時=2039.72s

KNN (自適應) 測試準確率: 0.998395
推理耗時: 121.35 秒
分類報告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     4976

In [14]:
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
print("特徵標準化...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print (X_test_scaled.shape, y_test)

特徵標準化...
(75383, 82)        Label
0          0
1          0
2          0
3          0
4          0
...      ...
75378      1
75379      1
75380      1
75381      1
75382      1

[75383 rows x 1 columns]


In [6]:
import pandas as pd
import numpy as np
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 載入資料
X_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_train_upsampled.csv')
Y_train = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/Y_train_upsampled.csv')
X_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/X_test_ext.csv')
y_test = pd.read_csv('/Users/User/.vscode/thesis_experiment/CIC-DDoS-2019/01-12/y_test_ext.csv')
y_train = Y_train.iloc[:, 0] if isinstance(Y_train, pd.DataFrame) else Y_train

# 特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class OptimizedRefinedAdaptiveKNN:
    def __init__(self, estimator, param_grid, n_iterations=3):
        self.estimator = estimator
        self.initial_param_grid = param_grid
        self.n_iterations = n_iterations
        self.best_params_ = None
        self.best_estimator_ = None
        self.best_score_ = 0
        self.optimization_history = []
        self.allowed_k = sorted(param_grid['n_neighbors'])
        self.sample_size = min(50000, len(X_train_scaled))  # 限制樣本大小以加速搜索
        
    def _get_refined_k_range(self, best_k, iteration):
        """動態調整k的搜索範圍，隨迭代變得更精細"""
        if iteration == 0:
            return self.allowed_k
        elif iteration == 1:
            min_k = max(min(self.allowed_k), best_k - 2)
            max_k = min(max(self.allowed_k), best_k + 2)
            refined_range = [k for k in self.allowed_k if min_k <= k <= max_k]
            return refined_range if refined_range else [best_k]
        else:
            min_k = max(min(self.allowed_k), best_k - 1)
            max_k = min(max(self.allowed_k), best_k + 1)
            refined_range = [k for k in self.allowed_k if min_k <= k <= max_k]
            return refined_range if refined_range else [best_k]

    def _generate_new_param_grid(self, best_params, iteration):
        """根據迭代次數生成不同精細度的參數網格"""
        new_grid = {}
        if 'n_neighbors' in best_params:
            best_k = best_params['n_neighbors']
            new_grid['n_neighbors'] = self._get_refined_k_range(best_k, iteration)
        if iteration == 0:
            for param in ['weights', 'p']:
                if param in self.initial_param_grid:
                    new_grid[param] = self.initial_param_grid[param]
        elif iteration == 1:
            if 'weights' in best_params:
                best_weight = best_params['weights']
                other_weights = [w for w in self.initial_param_grid['weights'] if w != best_weight]
                new_grid['weights'] = [best_weight] + other_weights[:1]
            if 'p' in best_params:
                best_p = best_params['p']
                other_p = [p for p in self.initial_param_grid['p'] if p != best_p]
                new_grid['p'] = [best_p] + other_p[:1]
        else:
            for param in ['weights', 'p']:
                if param in best_params:
                    new_grid[param] = [best_params[param]]
        return new_grid

    def _get_sample_indices(self, y, sample_size):
        """分層抽樣獲取訓練子集，確保類別平衡"""
        if len(y) <= sample_size:
            return np.arange(len(y))
        from sklearn.model_selection import train_test_split
        indices = np.arange(len(y))
        sample_indices, _ = train_test_split(
            indices, 
            train_size=sample_size, 
            stratify=y, 
            random_state=42
        )
        return sample_indices

    def fit(self, X, y):
        start_time = time.time()
        current_grid = self.initial_param_grid
        print("Begin KNN model of adaptive grid searchCV...")
        
        sample_indices = self._get_sample_indices(y, self.sample_size)
        
        for i in range(self.n_iterations):
            print(f"\niteration {i+1}/{self.n_iterations}")
            if i < self.n_iterations - 1:
                X_iter = X[sample_indices]
                y_iter = y.iloc[sample_indices] if hasattr(y, 'iloc') else y[sample_indices]
                cv_folds = 3
            else:
                X_iter = X
                y_iter = y
                cv_folds = 5
            
            skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
            
            gs = GridSearchCV(
                self.estimator, 
                current_grid, 
                cv=skf,
                scoring='accuracy',
                n_jobs=-1,
                error_score='raise',
                verbose=0
            )
            
            iter_start = time.time()
            gs.fit(X_iter, y_iter)
            iter_time = time.time() - iter_start
            
            iteration_best_params = gs.best_params_
            iteration_best_score = gs.best_score_
            
            if i == 0 or iteration_best_score > self.best_score_ or i == self.n_iterations - 1:
                self.best_params_ = iteration_best_params
                self.best_score_ = iteration_best_score
                self.best_estimator_ = self.estimator.set_params(**self.best_params_)
                if i < self.n_iterations - 1:
                    self.best_estimator_.fit(X, y)
                else:
                    self.best_estimator_ = gs.best_estimator_
            
            print(f"best parameter: {iteration_best_params}")
            print(f"best grade: {iteration_best_score:.6f}")
            print(f"execution time: {iter_time:.2f} sec")
            
            self.optimization_history.append({
                'iteration': i+1,
                'param_grid': current_grid,
                'best_params': iteration_best_params,
                'best_score': iteration_best_score,
                'iteration_time': iter_time
            })
            
            if i < self.n_iterations - 1:
                current_grid = self._generate_new_param_grid(iteration_best_params, i)
        
        total_time = time.time() - start_time
        print(f"\ntotal execution time: {total_time:.2f} sec")
        print(f"final best parameter: {self.best_params_}")
        print(f"final best grade: {self.best_score_:.6f}")
        
        print(f"\n the history of optimized parameter:")
        for hist in self.optimization_history:
            print(f"iteration{hist['iteration']}: best parameter={hist['best_params']}, "
                  f"grade={hist['best_score']:.6f}, execution time={hist['iteration_time']:.2f}s")
        
        return self, total_time

# 初始參數範圍
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

knn_model = KNeighborsClassifier()
optimized_knn_search = OptimizedRefinedAdaptiveKNN(knn_model, param_grid_knn, n_iterations=3)
optimized_knn_search, total_duration = optimized_knn_search.fit(X_train_scaled, y_train)

# 測試集評估
def evaluate_model(model, X_test, y_test, model_name="Model"):
    start = time.time()
    y_pred = model.predict(X_test)
    infer_time = time.time() - start
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{model_name} testing accuracy: {acc:.6f}")
    print(f"classification_report:\n{classification_report(y_test, y_pred)}")
    print(f"confusion_matrix:\n{confusion_matrix(y_test, y_pred)}")
    return acc

final_accuracy = evaluate_model(optimized_knn_search.best_estimator_, X_test_scaled, y_test, "KNN (adaptive)")
print(f"\ntotal execution time: {total_duration:.2f} sec")

Begin KNN model of adaptive grid searchCV...

iteration 1/3
best parameter: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
best grade: 0.999140
execution time: 87.43 sec

iteration 2/3
best parameter: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
best grade: 0.999140
execution time: 86.12 sec

iteration 3/3
best parameter: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
best grade: 0.999797
execution time: 2040.25 sec

total execution time: 2214.20 sec
final best parameter: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
final best grade: 0.999797

 the history of optimized parameter:
iteration1: best parameter={'n_neighbors': 3, 'p': 1, 'weights': 'distance'}, grade=0.999140, execution time=87.43s
iteration2: best parameter={'n_neighbors': 3, 'p': 1, 'weights': 'distance'}, grade=0.999140, execution time=86.12s
iteration3: best parameter={'n_neighbors': 3, 'p': 1, 'weights': 'distance'}, grade=0.999797, execution time=2040.25s

KNN (adaptive) testing accuracy: 0.998395
clas